# IUM projekt - Zniżki dla klientów

## Zadanie
- Zinterpretować: “Niektórzy  klienci  nie  mogą  zdecydować  się  na zakup oglądając  produkt  na  stronie – ale pewnie  gdybyśmy  wiedzieli,  które  to  są  osoby,  to odpowiednia zniżka  skłoniłaby ich do zakupu."  
- Analiza danych.  
- Doprecyzowanie, przeanalizowanie problemu i wdrozenie aplikacji bazującej na mikroserwisach.  

## Problem biznesowy
Stworzenie systemu, który umożliwi zaproponowanie klientom niezdecydowanym, zniżki takiej, która zachęci ich do zakupu, ale równocześnie bedzie jak najmniejsza.  
Istnieje kilka interpretacji zadania:  
1. Model ma mówić czy klient jest niezdecydowany
2. Model ma mówić czy klient jest niezdecydowany i jakoś optymalizować stawkę zniżki
3. Model ma mówić czy opłaca się zaproponować zniżkę klientowi
4. Model ma wyznaczać wartośc zniżki którą powinno się zaproponować każdemu klientówi na podstawie jego historii
5. Przewidywać, po ilu wyświetlonych produktach klient, będzie chciał kupić produkt.  

Zakładamy interpretację nr.4.

## Zadania modelowania
- zadanie grupowania ( grupujemy klientów, żeby wspomóc przewidywanie zniżki )
- zadanie regresji ( znalezienie takiej znizki, która spowoduje, że klient zdecyduje się na zakup, ale firma zarobi jak najwięcej )

## Założenia
* Będziemy na zbiorze testowym sprawdzać czy dobrze przewidzieliśmy, że klient dokona zakupu. To znaczy, że sprawdzamy historie klienta i jeżeli wyświetlał produkty a przy danej zniżce i dokonał zakupu, to oznacza, że nasz model również powinien to przewidzieć.
* Odpowiedna zniżka to taka, która zachęci klienta do zakupu. Przy czym musi ona przynieść jak najwiekszy zysk firmie. To znaczy, że znajdujemy pierwszą zniżkę dla której model przewidzi, że klient dokona zakupu.
* Zakładamy, że testów online nie będziemy mogli przeprowadzić, ograniczymy się wiec do estymacji jakości modelu.

## Kryteria sukcesu
- Stworzenie systemu, który przyniesie jak największy zysk firmie.  
Składa się na to zmotywowanie do zakupu jak największej ilości klientów, ale nie kosztem straty wynikającej z za dużej zniżki.  
- Chcemy, żeby nasz model miał jak największy stosunek *liczby zaoferowanych zniżek* / *sumy oferowanych zniżek*



## Poniżej wstępna analiza danych

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import random
import numpy as np
import pandas as pd

In [2]:
users = pd.read_json(path_or_buf='data/users.jsonl', lines=True)
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  200 non-null    int64 
 1   name     200 non-null    object
 2   city     200 non-null    object
 3   street   200 non-null    object
dtypes: int64(1), object(3)
memory usage: 6.4+ KB


In [3]:
deliveries = pd.read_json(path_or_buf='data/deliveries.jsonl', lines=True)
deliveries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3117 entries, 0 to 3116
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   purchase_id         3117 non-null   int64  
 1   purchase_timestamp  3117 non-null   object 
 2   delivery_timestamp  2962 non-null   object 
 3   delivery_company    2963 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 97.5+ KB


In [4]:
sessions = pd.read_json(path_or_buf='data/sessions.jsonl', lines=True)
sessions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56883 entries, 0 to 56882
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   session_id        56883 non-null  int64         
 1   timestamp         56883 non-null  datetime64[ns]
 2   user_id           54075 non-null  float64       
 3   product_id        54048 non-null  float64       
 4   event_type        56883 non-null  object        
 5   offered_discount  56883 non-null  int64         
 6   purchase_id       3117 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(1)
memory usage: 3.0+ MB


    
"RangeIndex: 56883 entries, 0 to 56882  
6   purchase_id       3117 non-null   float64 "  
  
purchase_id jest związane z zakupem więc występuje czesto jako null, gdy tylko wyświetlamy

In [5]:
products = pd.read_json(path_or_buf='data/products.jsonl', lines=True)
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319 entries, 0 to 318
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   product_id     319 non-null    int64  
 1   product_name   319 non-null    object 
 2   category_path  319 non-null    object 
 3   price          319 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 10.1+ KB


## Wykonujemy czyszczenie danych
Procedura opisana w pliku clean_data.ipynb

In [8]:
products_clean = pd.read_json(path_or_buf='data/products_clean.jsonl', lines=True)
products_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308 entries, 0 to 307
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_id    308 non-null    int64  
 1   product_name  308 non-null    object 
 2   category      308 non-null    object 
 3   price         308 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 9.8+ KB


In [9]:
sessions_clean = pd.read_json(path_or_buf='data/sessions_clean.jsonl', lines=True)
sessions_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53278 entries, 0 to 53277
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   session_id        53278 non-null  int64         
 1   timestamp         53278 non-null  datetime64[ns]
 2   user_id           53278 non-null  int64         
 3   product_id        53278 non-null  int64         
 4   event_type        53278 non-null  object        
 5   offered_discount  53278 non-null  int64         
 6   purchase_id       2937 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 2.8+ MB


## Tworzymy tabelę zawierającą ważne informację
Chcemy wykorzystać potencjalnie najbardziej wnoszące informację dla naszego problemu dane jako podstawę do tworzenia cech modelu.  
Dane w pliku deliveries odnoszą się do dostaw produktu więc nic nie wnoszą do określenia czy klient jest niezdecydowany.  
Dane w pliku users opisują klienta dokładniej, lecz są to dane demograficzne więc mało wnoszą do naszego problemu.

In [13]:
features_products = products_clean[['product_id','category','price']].copy()
features_sessions = sessions_clean[['session_id','timestamp','user_id','product_id','event_type','offered_discount']].copy()
features = pd.merge(features_sessions, features_products, on='product_id')
features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53278 entries, 0 to 53277
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   session_id        53278 non-null  int64         
 1   timestamp         53278 non-null  datetime64[ns]
 2   user_id           53278 non-null  int64         
 3   product_id        53278 non-null  int64         
 4   event_type        53278 non-null  object        
 5   offered_discount  53278 non-null  int64         
 6   category          53278 non-null  object        
 7   price             53278 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(4), object(2)
memory usage: 3.7+ MB


## Potencjalne cechy modelu klasyfikacji
Na podstawie wyżej wypisanych danych można wnioskować, że da się z nich utworzyć cechy: 
- ile razy dany klient przegladal ten produkt
- ile razy dany klient przegladal dana kategorie
- ile razy dany klient w tej sesji przegladal produkty z tej kategorii,
- ile razy znizka zachecila go do zakupu
- ile razy kupowal cos z tej kategorii
  
W tym celu wymagany bedzie dostęp do historii eventów klienta oraz dostęp do jego najnowszych akcji.  
Nasza aplikacja powinna przetworzyć i przechowywać niektóre cechy klienta, żeby poźniej sieć neuronowa mogła na podstawie nich wnioskować, jak bardzo niezdecydowany jest klient.
Kiedy nasza aplikacja dostanie informację o nowej akcji klienta, powinna już wiedzieć jak zachowywał się w przeszłości i uwzględnić to w przewidywaniu.

## Labelowanie danych
Do uczenia naszej sieci neuronowej będziemy musieli ocenić w jakiś sposób którzy klienci byli niezdecydowani.  
Najbardziej wymiernym sposobem byłoby dodanie do strony ankiety, w której dany klient mógłby podać jak bardzo niezdecydowany jest co do zakupu. (np. w skali od 1-6)  
Dzięki temu nasza się uczyłaby się rozpoznawać to jak się czują faktyczni klienci i po jakimś czasie nie musielibyśmy pytać ich oto w ankiecie.  
  
  
Drugą opcją jest automatyczne labelowanie dotychczasowych danych. W tym celu należało będzie utworzyć funkcję labelującą czy dany klient, na podstawie dotychczasowych danych, jest niezdecydowany czy nie. Będzie to wymagało niestety pewnej dozy heurystyki, więc nasze nasza sieć będzie się uczyć pewnego rodzaju schematu. Możnaby próbować to zredukować wprowadzając uczenie online, które będzie się douczało po wdrożeniu sieci neuronowej. 

## Dane potrzebne do modelu regresji
Do przewidzenia jaką zniżkę powinien otrzymać dany klient będziemy potrzebowali wiedzieć czy nasze dotychczasowe zniżki przynoszą zysk firmie, jeżeli nie to powinniśmy dawać mniejsze zniżki. Powinniśmy, więc mieć dostęp do jakiejś statystyki przychodu firmy.  
Przydatną informacją będzie też moment, w którym klient jest niezdecydowany, dzięki czemu jeżeli zbliżają się róznego rodzaju święta to można zwiększyć szansę na większą zniżkę.
W tym modelu będziemy korzystać z przewidywań modelu klasyfikacji, który zwróci nam stopień niezdecydowania klienta.